In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import make_scorer, mean_squared_error

RANDOM_STATE = 42

In [2]:
# Load the processed training data
df_train = pd.read_csv('../data/train_processed.csv')
features = ['Therapy Hours', 'Initial Health Score', 'Lifestyle Activities', 'Average Sleep Hours', 'Follow-Up Sessions']
target = 'Recovery Index'

X = df_train[features]
y = df_train[target]

print(f"Features (X) shape: {X.shape}")
print(f"Target (y) shape: {y.shape}")

Features (X) shape: (8000, 5)
Target (y) shape: (8000,)


In [ ]:
# --- Model 5: Decision Tree Regressor ---

print("Starting Model 5: Decision Tree Tuning...")

#Initialize the model
model_dt = DecisionTreeRegressor(random_state=RANDOM_STATE)

#Define the parameters to search
param_grid = {
    'max_depth': [3, 5, 7, 10, None],
    'min_samples_leaf': [5, 10, 20, 50]
}

#Set up GridSearchCV for RMSE
search_rmse = GridSearchCV(
    model_dt,
    param_grid,
    cv=10,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

search_rmse.fit(X, y)

print("\n--- Decision Tree Results (RMSE) ---")
print(f"Best Parameters found: {search_rmse.best_params_}")
best_rmse = np.sqrt(-search_rmse.best_score_)
print(f"Average 10-Fold RMSE (from best model): {best_rmse:.3f}")

#Set up GridSearchCV for R-squared
search_r2 = GridSearchCV(
    model_dt,
    param_grid,
    cv=10,
    scoring='r2',
    n_jobs=-1
)
search_r2.fit(X, y)

print("\n--- Decision Tree Results (R-squared) ---")
print(f"Average 10-Fold R-squared (from best model): {search_r2.best_score_:.3f}")

Starting Model 4: Decision Tree Tuning...

--- Decision Tree Results (RMSE) ---
Best Parameters found: {'max_depth': None, 'min_samples_leaf': 10}
Average 10-Fold RMSE (from best model): 2.429

--- Decision Tree Results (R-squared) ---
Average 10-Fold R-squared (from best model): 0.984


In [4]:
# --- Decision Tree Submission Cell ---

print("Loading test data...")
X_test = pd.read_csv('../data/test_processed.csv')
test_ids = pd.read_csv('../data/test_ids.csv')
print("Test data and IDs loaded.")

#Get the best parameters from the search
best_params = search_rmse.best_params_
print(f"\nTraining a new model with best params: {best_params}")

final_dt_model = DecisionTreeRegressor(random_state=RANDOM_STATE)

final_dt_model.set_params(**best_params)

final_dt_model.fit(X, y)
print("Model trained successfully.")

print("\nGenerating predictions...")
predictions = final_dt_model.predict(X_test)

print("Creating submission file...")
submission_df = pd.DataFrame({'Id': test_ids['Id'], 'Recovery Index': predictions})
submission_df['Recovery Index'] = submission_df['Recovery Index'].round().astype(int)

submission_path = '../submission/submission_model5.csv'
submission_df.to_csv(submission_path, index=False)

print(f"\nDecision Tree submission file saved to: {submission_path}")
submission_df.head()

Loading test data...
Test data and IDs loaded.

Training a new model with best params: {'max_depth': None, 'min_samples_leaf': 10}
Model trained successfully.

Generating predictions...
Creating submission file...

Decision Tree submission file saved to: ../submission/submission_model5.csv


,Id,Recovery Index
0,6253,56
1,4685,23
2,1732,48
3,4743,30
4,4522,43
